# Views

>Views are database objects that take the form of virtual tables, acting like shortcuts to regularly used queries. Queries can be saved as views, so that rather than repeatedly typing out the same query structure, you can refer to the view instead. Once created, views can be used in any subsequent query. They provide a way to encapsulate frequently-used logic into reusable elements, making your queries more DRY. 

Views are dynamic: they do not store any data directly, but instead are representations of the tables from which they are created. They don’t change the size of your data model, and if you update the date in the underlying table, the view will also update accordingly. The exception to this rule is a special type of view called a *Materialized View*, which will be covered later in this lesson.

## Properties of Views

Like other tables, you can:

- Query a view
- `JOIN` views to regular tables, or to other views
- Perform typical query operations like filtering and aggregations
- Perform **CRUD** operations via the view onto the underlying tables

Common use-cases of views:

- **Avoid Effort Duplication:** Write a query once, and store it as a view for referencing in future queries
- **Reduce Complexity:** Improve readability and use for yourself or other database users by showing only the columns relevant for a particular task
- **Increase Security:** Views can be used to limit user access to specific subsets of a database by granting permissions for users to use specific views, but not the underlying tables

## Creating Views

The syntax for creating a view is as follows:


```sql
CREATE VIEW view_name AS {query};
```


You can also overwrite an existing view with the following syntax:


```sql
REPLACE VIEW view_name AS {query};
```

These two pieces of syntax are usually combined into a single statement which will either create or update the view depending on whether it already exists:

```sql
CREATE OR REPLACE VIEW view_name AS {query};
```

As a simple example, you might create a view called `movie_list` from the `film` table in the **Pagila** database, containing just the `film_id` (primary key), `title` and `rating` of the film, as follows:


In [ ]:


CREATE OR REPLACE VIEW movie_list AS
SELECT 
    film_id,
    title,
    rating
FROM 
    film;




After creating the view, you can then query it just like any other table:


In [ ]:

SELECT title, rating FROM movie_list
WHERE rating = 'PG-13'
ORDER BY title DESC
LIMIT 5;


<p align="center">
    <img src="images/first_query_result.jpg"  width="700"/>
</p>
<br>

Typically, views are created for more complex queries, for example by joining tables and performing aggregations. This can simplify future queries by removing the need for Common Table Expressions (CTEs) and subqueries. For example the following SQL statement creates a view of the total revenue per customer for the DVD rental business in the **Pagila** database, by joining the `customer` and `payment` tables and aggregating the `payment_amount` as a sum:

In [ ]:
CREATE OR REPLACE VIEW customer_payments AS
SELECT 
    customer.customer_id,
    customer.first_name,
    customer.last_name,
    SUM(payment.amount) AS total_paid
FROM 
    customer
JOIN 
    payment ON customer.customer_id = payment.customer_id
GROUP BY 
    customer.customer_id, customer.first_name, customer.last_name;

<p align="center">
    <img src="images/second_query_result.jpg"  width="700"/>
</p>
<br>

## View Management



To rename a view, the syntax is:

```sql
ALTER VIEW view_name RENAME TO new_view_name;
```

To delete a view, use the following syntax:

```sql
DROP VIEW view_name;
```

## Materlialized Views

>Unlike standard views, materialized views **do** store the underlying data that was used to create the view. They are used to cache the results of a query, which can be used to improve performance for expensive queries that are used regularly. 

They can also be used to create permanent, time-dependent aggregations of data, such as calculating monthly or yearly sales data. Whereas regular views update dynamically with the tables that were used to generate them, materialized views maintain a snapshot of the data as it was at the moment the view was created. In contrast to standard views, materlialized views increase the size of the data model.

Materialized views can be created as follows:

```sql
CREATE MATERIALIZED VIEW IF NOT EXISTS view_name AS 
{query}
WITH [NO] DATA; 
```

The `WITH DATA` option is used when you intend to populate the materialized view with data at the time of its creation, whereas the `WITH NO DATA` option can be used it you want to define the view but don’t yet want to populate it with any data.

The following materialized view counts the total number of actors in each film, but does not yet populate it with any data:

In [ ]:
CREATE MATERIALIZED VIEW film_actor_count AS
SELECT 
    film.film_id,
    film.title,
    COUNT(film_actor.actor_id) AS actor_count
FROM 
    film
JOIN 
    film_actor ON film.film_id = film_actor.film_id
GROUP BY 
    film.film_id, film.title
WITH NO DATA;

When we attempt to query the view:

```sql
SELECT * FROM film_actor_count;
```

Nothing is immediately returned, because there is no data in the view yet.

<p align="center">
    <img src="images/empty_view.jpg"  width="700"/>
</p>
<br>

We can update (*refresh*) the data in a materialized view with the following syntax:


```sql
REFRESH MATERIALIZED VIEW film_actor_count;
```


After running this statement, the `SELECT` statement returns data:

<p align="center">
    <img src="images/third_query_result.jpg"  width="700"/>
</p>
<br>

The decision of whether to use a standard view or a materialized view is a tradeoff, between execution time and data model size on the one hand, and data freshness on the other. If it is more important that a query run quickly, a materialized view is better. If it is more important that the data are current, then a standard view is preferable.

## Key Takeaways

- **Dynamic Representation**: Views act as virtual tables, representing queries that reflect the current state of the underlying tables, but do not store data themselves

- **DRY Principle**: By using views, you encapsulate queries you use frequently, adhering to the DRY principle, thereby reducing redundancy in database operations

- **View Management**: Operations like renaming and deleting views are supported, allowing effective management of views within the database environment

- **Simplified Querying**: Views allow users to execute complex queries easily by abstracting the underlying complexity into a simpler interface, improving readability and manageability

- **Enhanced Security**: Views can restrict access to data by exposing only specific fields or records, thus enhancing database security through controlled visibility.

- **SQL Syntax for Views**: The `CREATE OR REPLACE VIEW` command is used to define views

- **Join and CRUD Operations**: Views can be joined with other views or tables and can be used for CRUD operations, offering flexible interaction with the database

- **Materialized Views**: Unlike standard views, materialized views store query results and can improve performance for complex, frequently executed queries by caching the data